/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
import numpy as np
import sys
import datetime
import time
import pandas as pd
import math
import glob
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier

# left_eyes_points_list = [18, 19, 20, 21, 22, 37, 39, 39, 40, 41, 42]
# right_eyes_points_list = [23, 24, 25, 26, 27, 43, 44, 45, 46, 47, 48]
# mouth_points_list = [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]

class smile_fv:
    '''
    classdocs
    '''


    def __init__(self, segments, fold_list_number, position_list):
        '''
        Constructor
        '''
        
        '''
        AU6 left eye points:
        '''
        self.left_eyes_points_list = [18, 19, 20, 21, 22, 37, 39, 39, 40, 41, 42]
        self.right_eyes_points_list = [23, 24, 25, 26, 27, 43, 44, 45, 46, 47, 48]
        self.mouth_points_list = [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
        self.left_eyes_feature_list = []
        self.right_eyes_feature_list = []
        self.mouth_feature_list = []
        self.segments = segments
        self.fold_list_number = fold_list_number
        self.position_list = self.__get_points__(position_list)
        self.position = position_list
        self.feature_values = None
        self.frames = 0
        # print(self.position_list)
    
    
    def __get_points__(self, position_list):
        feature_list = []
        if (position_list == 'left_eye'):
            for i in range(0,len(self.left_eyes_points_list)):
                for j in range(i + 1,len(self.left_eyes_points_list)):
                    feature_list.append([self.left_eyes_points_list[i], self.left_eyes_points_list[j]])
            self.left_eyes_feature_list = feature_list
            
        elif (position_list == 'right_eye'):
            for i in range(0,len(self.right_eyes_points_list)):
                for j in range(i + 1,len(self.right_eyes_points_list)):
                    feature_list.append([self.right_eyes_points_list[i], self.right_eyes_points_list[j]])
            self.right_eyes_points_list = feature_list
                    
        elif (position_list == 'mouth'):
            for i in range(0,len(self.mouth_points_list)):
                for j in range(i + 1,len(self.mouth_points_list)):
                    feature_list.append([self.mouth_points_list[i], self.mouth_points_list[j]])
            self.mouth_points_list = feature_list
        else:
            self.position_list = position_list
        return feature_list


    def read_file(self, filename):
        # print('Reading file ' + filename)
        self.feature_values = pd.read_csv(filename)
        self.frames = self.feature_values.shape[0]
        # self.feature_values.drop([0], axis=1)


    def __distances__(self, x1s, x2s, y1s, y2s):
        distances = []
        for x1, x2, y1, y2 in zip(x1s, x2s, y1s, y2s):
            distances.append(math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1)))
        return distances


    def get_feature_vector_segment(self, segment):
        if segment < 0 or segment >= self.segments:
            return []
        segment_fv = []
        start = int(float(segment) / self.segments * self.frames)
        end = int(float((segment + 1)) / self.segments * self.frames)
        # Extract features
        for feature_pair in self.position_list:
            # print('Generating features for pair ' + str(feature_pair))
            x1_segment_label = str(feature_pair[0]) + '_x'
            x2_segment_label = str(feature_pair[1]) + '_x'
            y1_segment_label = str(feature_pair[0]) + '_y'
            y2_segment_label = str(feature_pair[1]) + '_y'
            
            x1_segment = self.feature_values.loc[start:end, x1_segment_label]
            x2_segment = self.feature_values.loc[start:end, x2_segment_label]
            y1_segment = self.feature_values.loc[start:end, y1_segment_label]
            y2_segment = self.feature_values.loc[start:end, y2_segment_label]
            distances = self.__distances__(x1_segment, x2_segment, y1_segment, y2_segment)
            segment_fv.extend([np.mean(distances), np.std(distances)])
        return segment_fv


    def get_feature_vector(self, filename):
        self.read_file(filename)
        feature_vector = []
        for segment in range(self.segments):
            feature_vector.extend(self.get_feature_vector_segment(segment))
        return feature_vector
    
    def load_fold_list(self, fold_list_number):
        list_path = '/Users/sam/Documents/USF/homework/cs686-02/HCI/lab/final_project/file-video-stream/SMILE_DATABASE/experimental_protocols/fold_all'
        allFiles = glob.glob(list_path + "/*.txt")
        whole_fold_list = []
        train_fold_list = []
        test_fold_list = []
        i = 1

        while i < 11:

            file_ = list_path + "/fold_all_" + str(i) + ".txt"

            with open(file_) as f:
                content = f.readlines()
                content = [x.strip() for x in content]
                whole_fold_list.append(content)
            i+=1

        test_fold_list = whole_fold_list[fold_list_number]
        for x in whole_fold_list:
            if x != test_fold_list:
                train_fold_list.extend(x)

        return train_fold_list,test_fold_list
    
    def read_data(self):
        # deli_training_path = '/Users/sam/Documents/USF/homework/cs686-02/HCI/lab/final_project/file-video-stream/videos/deliberate/results'
        """
        if 'deliberate' in directory_name:
            label = 0
        else:
            label = 1
        """
        training_fold_list,test_fold_list = self.load_fold_list(self.fold_list_number)
        # print (training_fold_list)
        #  we need to load spontaneous and deliberate seperately
        deli_training_path = 'file-video-stream/videos/deliberate/new_results'
        deli_allFiles = glob.glob(deli_training_path + "/*.csv")
        deli_training_df = pd.DataFrame()
        deli_test_df = pd.DataFrame()
        deli_training_list_ = []
        deli_test_list_ = []
        count_deli = 0
        count_spon = 0
        for file_ in deli_allFiles:
            count_deli +=1
            #print("deli is going on the %d th:" % count_deli)
            feature_vector_list = self.get_feature_vector(file_)
            df = pd.DataFrame([feature_vector_list])
            
            label = 0
            file_name = file_[len(deli_training_path)+1 :len(deli_training_path)+23]
            # print(file_name)
            if file_name in training_fold_list:
                deli_training_list_.append(df)
            else:
                deli_test_list_.append(df)
#         print ("the length of deli trainging list is: %d" %(len(deli_training_list_)))
#         print ("the length of deli test list is: %d" %(len(deli_test_list_)))
        deli_training_df = pd.concat(deli_training_list_)
        deli_test_df = pd.concat(deli_test_list_)
        deli_training_df['Label'] = 0
        deli_test_df['Label'] = 0
        print (deli_training_df.shape[0])
        print (deli_test_df.shape[0])
        
        spon_training_path = 'file-video-stream/videos/spontaneous/new_results'
        spon_allFiles = glob.glob(spon_training_path + "/*.csv")
        spon_training_df = pd.DataFrame()
        spon_test_df = pd.DataFrame()
        spon_training_list_ = []
        spon_test_list_ = []

        for file_ in spon_allFiles:
            count_spon +=1
            #print("spon is going on the %d th:" % count_spon)
            feature_vector_list = self.get_feature_vector(file_)
            df = pd.DataFrame([feature_vector_list])

            label = 1
            file_name = file_[len(spon_training_path)+1 :len(spon_training_path)+24]
            # print(file_name)
            if file_name in training_fold_list:
                spon_training_list_.append(df)
            else:
                spon_test_list_.append(df)

        spon_training_df = pd.concat(spon_training_list_)
        spon_test_df = pd.concat(spon_test_list_)
        spon_training_df['Label'] = 1
        spon_test_df['Label'] = 1
        
        print (spon_training_df.shape[0])
        print (spon_test_df.shape[0])
        
        traing_list = [spon_training_df, deli_training_df]
        traing_df = pd.concat(traing_list)
        
        test_list = [spon_test_df, deli_test_df]
        test_df = pd.concat(test_list)
        
        return traing_df, test_df
    
    def get_feature_pair(self,feature_index):
        feature_pair_index = 0
        period = 0
        if (self.position == 'left_eye' or self.position == 'right_eye'):
            feature_pair_index = feature_index % 55
            period = feature_index // 110
        elif(self.position == 'mouth'):
            feature_pair_index = feature_index % 190
            period = feature_index //380
        return self.position_list[feature_pair_index], period
        
# def main():
#     # Your code here for main function


# if __name__ ==  '__main__':
#     main()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
smile_fv_left_eye = smile_fv(4, 0, 'left_eye')
left_eye_traing_df, left_eye_test_df = smile_fv_left_eye.read_data()

577
66
535
62


In [3]:
left_eye_whole_list = [left_eye_traing_df, left_eye_test_df]
left_eye_whole_df = pd.concat(left_eye_whole_list)
# add one random column to it
left_eye_whole_df['random'] = np.random.uniform(0, 1, size=left_eye_whole_df.shape[0])

# left_eye_whole_df = left_eye_whole_df[[145, 221,89,29,105,327,337,257,417,325,367,121,6,85,153,241,31,146,199,299, 'random', 'Label']]
# left_eye_whole_df.head()

In [4]:
left_eye_whole_df.head()

,0,1,2,3,4,5,6,7,8,9,...,432,433,434,435,436,437,438,439,Label,random
0,8.867597,0.594050,17.119188,1.148048,24.559673,1.387453,30.755322,1.071753,14.020392,0.472735,...,7.049426,0.597950,5.781765,0.469626,11.511456,0.462238,5.836130,0.400462,1,0.106282
0,6.512064,0.391189,13.052767,0.453070,19.531024,0.647392,25.991675,0.807966,10.616766,0.494256,...,5.730266,0.352209,4.728671,0.468686,9.523322,0.496321,4.839269,0.376136,1,0.807107
0,8.443402,0.394891,16.636206,0.419559,24.570742,0.582462,32.746213,0.761213,13.723313,0.547228,...,5.884406,0.531189,5.337944,0.465062,10.133572,0.522057,4.833416,0.415255,1,0.914991
0,6.457860,0.435501,13.164410,0.501497,19.855873,0.504906,26.391023,0.487157,11.769883,0.388220,...,6.532947,0.527299,5.025282,0.043177,10.009551,0.019625,5.006320,0.024205,1,0.544506
0,8.443393,0.457287,16.745149,0.516371,24.995729,0.665345,33.665475,0.699957,15.135466,0.409487,...,7.178936,0.461386,6.027277,0.388709,11.907545,0.435352,5.953018,0.248147,1,0.915769


In [5]:
# cols = list(whole_df.columns.values)
left_eye_whole_df = left_eye_whole_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 'random', 'Label']]
left_eye_whole_df.head()

,0,1,2,3,4,5,6,7,8,9,...,432,433,434,435,436,437,438,439,random,Label
0,8.867597,0.594050,17.119188,1.148048,24.559673,1.387453,30.755322,1.071753,14.020392,0.472735,...,7.049426,0.597950,5.781765,0.469626,11.511456,0.462238,5.836130,0.400462,0.106282,1
0,6.512064,0.391189,13.052767,0.453070,19.531024,0.647392,25.991675,0.807966,10.616766,0.494256,...,5.730266,0.352209,4.728671,0.468686,9.523322,0.496321,4.839269,0.376136,0.807107,1
0,8.443402,0.394891,16.636206,0.419559,24.570742,0.582462,32.746213,0.761213,13.723313,0.547228,...,5.884406,0.531189,5.337944,0.465062,10.133572,0.522057,4.833416,0.415255,0.914991,1
0,6.457860,0.435501,13.164410,0.501497,19.855873,0.504906,26.391023,0.487157,11.769883,0.388220,...,6.532947,0.527299,5.025282,0.043177,10.009551,0.019625,5.006320,0.024205,0.544506,1
0,8.443393,0.457287,16.745149,0.516371,24.995729,0.665345,33.665475,0.699957,15.135466,0.409487,...,7.178936,0.461386,6.027277,0.388709,11.907545,0.435352,5.953018,0.248147,0.915769,1


In [6]:
features = left_eye_whole_df.columns[0:441]
print(features)


Index([       0,        1,        2,        3,        4,        5,        6,
              7,        8,        9,
       ...
            431,      432,      433,      434,      435,      436,      437,
            438,      439, 'random'],
      dtype='object', length=441)


In [7]:
X_whole = left_eye_whole_df[features]
y_whole = left_eye_whole_df['Label']

In [8]:
feature_importance_list =[]
for i in range(1,11):
    print(i)
    clf = DecisionTreeClassifier()
    clf.fit(X_whole, y_whole)
    for j in range(0,len(clf.feature_importances_) - 1):
        if feature_importance_list == []:
            feature_importance_list = clf.feature_importances_
        else:
            feature_importance_list[j] += clf.feature_importances_[j]
feature_importance_list = [x / 10 for x in feature_importance_list]
print(feature_importance_list)

1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  import sys


2
3
4
5
6
7
8
9
10
[0.0, 0.00069988789636796374, 0.0, 0.00088139728487657851, 0.00082986707712201448, 0.003478732739343726, 0.0, 0.00073757416771085406, 0.00024226888720429513, 0.00072680666161288544, 0.0, 0.0093402157055748926, 0.0, 0.0061958076962625895, 0.0, 0.0, 0.0015284492240074891, 0.0020189073933691258, 0.00094754053662124351, 0.00024226888720429513, 0.0, 0.012938037855911277, 0.00032302518293906023, 0.00029072266464515422, 0.00056067942467279749, 0.0022363281895781085, 0.00030507933944244581, 0.0, 0.00030507933944244581, 0.018612342668847946, 0.0, 0.023131606653867216, 0.0, 0.0019004648262914709, 0.0, 0.012303645294306231, 0.0, 0.0017921177602701424, 0.0, 0.0011809779471491256, 0.0, 0.0, 0.0, 0.0042495553469943294, 0.0, 0.0006068957982491432, 0.0, 0.00031293064597221452, 0.00048453777440859037, 0.00029109395795887767, 0.00086229778001232463, 0.00055375745646696025, 0.0039337288944578889, 0.0, 0.00052760779880046493, 0.0036737947320351324, 0.0, 0.0011946166475651453, 0.00183944

In [9]:
sort_ = (np.array(feature_importance_list)).argsort()[::-1][:20]
print(sort_)
for i in range(0, len(sort_)-1):
    top_one, period = smile_fv.get_feature_pair(sort_[i])
    print(top_one)
    print(feature_importance_list[sort_[i]])
    # print("%s %d" %(str(feature_importance_list[sort_[i]]), period))

[145 221  89  31 257 325 105 337 123  29 367  85 405 153 146  69 417 389
 299  21]


TypeError: get_feature_pair() missing 1 required positional argument: 'feature_index'

In [19]:
smile_fv_ = smile_fv(4, 0, 'left_eye')
traing_df, test_df = smile_fv_.read_data()

whole_list = [traing_df, test_df]
whole_df = pd.concat(whole_list)
# add one random column to it
whole_df['random'] = np.random.uniform(0, 1, size=whole_df.shape[0])
whole_df = whole_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 'random', 'Label']]
whole_df.head()

features = whole_df.columns[0:441]

X_whole = whole_df[features]
y_whole = whole_df['Label']

feature_importance_list =[]
for i in range(1,11):
    print(i)
    clf = DecisionTreeClassifier()
    clf.fit(X_whole, y_whole)
    for j in range(0,len(clf.feature_importances_) - 1):
        if feature_importance_list == []:
            feature_importance_list = clf.feature_importances_
        else:
            feature_importance_list[j] += clf.feature_importances_[j]
feature_importance_list = [x / 10 for x in feature_importance_list]

sort_ = (np.array(feature_importance_list)).argsort()[::-1][:]
print(sort_)
for i in range(0, len(sort_)-1):
    top_one = smile_fv_.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
        print(top_one)
        print(feature_importance_list[sort_[i]])

577
66
535
62
1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


2
3
4
5
6
7
8
9
10
[145 221  89  29 105 325 327 337 257 367  31 153 417  85 146  69 199 299
 347 399 121 117 123 255  35 241  21 369 389 385 333  11 393 391 215 165
 405 285 189 233 297  70 125 345  95  81 175 435 135 346 395 216 303 197
 231 101 111  13  55  73 203 373 159 184 187 377 436  71 157  94 387 374
 313  74 225 130 375 163 301  52 407 319  66 219 227 155 403 124 209 177
  64 174 162 429 302 361 119 356 349 351  72 342 368 343 353 413 127 376
 381 100 223 210   1 409 286 437 179 340  16 295   3 172  57   5 390  83
 253 380 265 423 315 264 329 176 185  43 212 180 402 396 294 242 317 354
  58 293 137 321 141 364  19 133 357 154  49 262  76  50 244 205 323 359
 120 348  17 183  51 129 372 433 182 400 305 237 379  36 290 287  68 161
 382  39  93 113 339 156  84 418 115 291 211 318 147 112  75 152  24 206
 398 431 408 266 422 178 247 341 220 289 168 217  45  41 292 430 131 394
 414  77 128  59  40 164   0 332 222  20  22  33 283 279  97 383 411 271
 213 338 109 158 263 208 181  37

In [24]:
for i in range(0, len(sort_)-1):
    top_one,period = smile_fv_.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
        print(period)
        # print(top_one)
        # print(feature_importance_list[sort_[i]])

1
2
0
0
0
2
2
3
2
3
0
1
3
0
1
0
1
2
3
3
1
1
1
2
0
2
0
3
3
3
3
0
3
3
1
1
3
2
1
2
2
0
1
3
0
0
1
3
1
3
3
1
2
1
2
0
1
0
0
0
1
3
1
1
1
3
3
0
1
0
3
3
2
0
2
1
3
1
2
0
3
2
0
1
2
1
3
1
1
1
0
1
1
3
2
3
1
3
3
3
0
3
3
3
3
3
1
3
3
0
2
1
0
3
2
3
1
3
0
2
0
1
0
0
3
0
2
3
2
3
2
2
2
1
1
0
1
1
3
3
2
2
2
3
0
2
1
2
1
3
0
1
3
1
0
2
0
0
2
1
2
3
1
3
0
1
0
1
3
3
1
3
2
2
3
0
2
2
0
1
3
0
0
1
3
1
0
3
1
2
1
2
1
1
0
1
0
1
3
3
3
2
3
1
2
3
2
2
1
1
0
0
2
3
1
3
3
0
1
0
0
1
0
3
2
0
0
0
2
2
0
3
3
2
1
3
0
1
2
1
1
0
2
0
1
0
1
1
3
0
1
0
2
1
0
2
3
1
2
2
0
0
3
0
1
3
2
2
2
2
0
2
0
3
2
3
3
1
1
3
1
0
2
0
2
0
3
1
0
3
2
0
0
3
1
0
1
0
1
2
0
1
2
3
3
3
3
2
2
3
0
1
0
2
3


In [26]:
smile_fv_ = smile_fv(4, 0, 'right_eye')
traing_df, test_df = smile_fv_.read_data()

whole_list = [traing_df, test_df]
whole_df = pd.concat(whole_list)
# add one random column to it
whole_df['random'] = np.random.uniform(0, 1, size=whole_df.shape[0])
whole_df = whole_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 'random', 'Label']]
whole_df.head()

features = whole_df.columns[0:441]

X_whole = whole_df[features]
y_whole = whole_df['Label']

feature_importance_list =[]
for i in range(1,11):
    print(i)
    clf = DecisionTreeClassifier()
    clf.fit(X_whole, y_whole)
    for j in range(0,len(clf.feature_importances_) - 1):
        if feature_importance_list == []:
            feature_importance_list = clf.feature_importances_
        else:
            feature_importance_list[j] += clf.feature_importances_[j]
feature_importance_list = [x / 10 for x in feature_importance_list]

sort_ = (np.array(feature_importance_list)).argsort()[::-1][:]
print(sort_)
for i in range(0, len(sort_)-1):
    top_one = smile_fv_.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
#     print(period)
#     print(top_one)
        print(feature_importance_list[sort_[i]])

577
66
535
62
1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


2
3
4
5
6
7
8
9
10
[ 59 231 149 427 147 399 404 385 199 409 127 323 309 423 105  33 171 407
 175  11 201 142  71   5   1 237 245 267 367 285 113 365 165 434   9  75
 311 379 295  55 339 151 282 413 353  96 220 437 253 384 261  85 363 387
 331 241 210 271 160 180 225 273 188  27  97 258 395 155  25  29  61 196
 342 205 289 302 329 218  47 317  79 182  31 190 159 393   7 325 247 308
 173 320 226  73 293 129  39  30  99 397  13 212 229  19 191 376  41 137
 161  77 303 426 203 294 163 316 153 154 204  80 156   3 392 230 200 269
  65 209 415 115 178  38 341  22 313 206 430 297 211 410 288 111 193 351
 168 343  51  83 179 319 108 121 296 373 340 270 181 126 219 424 429 145
 322  89 292 432 419 236  45 305  91 120 281 133 106 187 213 356 438 278
 144 104 314 403  20 221 349 109 398 214 102 347 174 103 354 223 251 301
  98 143   8 318 227 428  62 337 355 338 307 169 257   4  24  56 197 141
  74 304 418  93  58  86 249 328  44 299 352 414 100 326 389 383 118 333
 315 369 101  68  60 346 330   6

In [30]:
for i in range(0, len(sort_)-1):
    top_one, period = smile_fv_.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
        print(period)
#         print(top_one)
#         print(feature_importance_list[sort_[i]])

0
2
1
3
1
3
3
3
1
3
1
2
2
3
0
0
1
3
1
0
1
1
0
0
0
2
2
2
3
2
1
3
1
3
0
0
2
3
2
0
3
1
2
3
3
0
2
3
2
3
2
0
3
3
3
2
1
2
1
1
2
2
1
0
0
2
3
1
0
0
0
1
3
1
2
2
2
1
0
2
0
1
0
1
1
3
0
2
2
2
1
2
2
0
2
1
0
0
0
3
0
1
2
0
1
3
0
1
1
0
2
3
1
2
1
2
1
1
1
0
1
0
3
2
1
2
0
1
3
1
1
0
3
0
2
1
3
2
1
3
2
1
1
3
1
3
0
0
1
2
0
1
2
3
3
2
1
1
1
3
3
1
2
0
2
3
3
2
0
2
0
1
2
1
0
1
1
3
3
2
1
0
2
3
0
2
3
0
3
1
0
3
1
0
3
2
2
2
0
1
0
2
2
3
0
3
3
3
2
1
2
0
0
0
1
1
0
2
3
0
0
0
2
2
0
2
3
3
0
2
3
3
1
3
2
3
0
0
0
3
3
0
2
1
2
3
3
2
0
1
3
3
1
2
2
0
3
1
3
2
2
3
2
0
0
3
3
0
2
2
2
1
2
1
0
1
1
0
1
1
3
2
1
0
3
3
2
3
2
0
0
0
0
1
0
1
2
3
3
2
2
1
2
0
0
1
0
1
2
1
0
3
0
1
1
3
0
1
0
0
1
0
1


In [7]:
smile_fv = smile_fv(4, 0, 'mouth')
traing_df, test_df = smile_fv.read_data()

whole_list = [traing_df, test_df]
whole_df = pd.concat(whole_list)


577
66
535
62


In [9]:
whole_df['random'] = np.random.uniform(0, 1, size=whole_df.shape[0])
whole_df = whole_df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 'random', 'Label']]
# whole_df.head()

features = whole_df.columns[0:1521]
X_whole = whole_df[features]
y_whole = whole_df['Label']

feature_importance_list =[]
for i in range(1,11):
    print(i)
    clf = DecisionTreeClassifier()
    clf.fit(X_whole, y_whole)
    for j in range(0,len(clf.feature_importances_) - 1):
        if feature_importance_list == []:
            feature_importance_list = clf.feature_importances_
        else:
            feature_importance_list[j] += clf.feature_importances_[j]
feature_importance_list = [x / 10 for x in feature_importance_list]

sort_ = (np.array(feature_importance_list)).argsort()[::-1][:]
print(sort_)
for i in range(0, len(sort_)-1):
    top_one, period = smile_fv.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
        print(period)
#         print(top_one)
        print(feature_importance_list[sort_[i]])

1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


2
3
4
5
6
7
8
9
10
[ 215 1496  493 ...,  941  942  760]
0
0.0856097596319
3
0.0619313347355
1
0.0359079801896
2
0.0302798403662
2
0.0261378252422
0
0.0208601773233
2
0.0204279146567
0
0.0187544526265
0
0.0182546364717
3
0.0177808755538
0
0.0173803621151
3
0.0158271993394
0
0.0155547313918
0
0.0151427704626
0
0.0148030336256
2
0.0146700705032
2
0.0145717008578
0
0.0138828194186
2
0.0138109642993
1
0.0137625996574
1
0.0134148727846
3
0.013255335347
1
0.0127895845135
0
0.0126620196511
2
0.0124295964513
3
0.0122143897299
1
0.0118508222603
1
0.0116808213473
0
0.0115585420845
3
0.0113032517726
2
0.0101564501269
2
0.0100781570064
2
0.00962393992729
2
0.0094260376558
1
0.00888454200505
1
0.00852786482959
2
0.00843950291447
2
0.00841875015699
3
0.00833159041766
0
0.00822857210907
2
0.00814398455121
0
0.00808255636995
0
0.00800179524595
1
0.00778244796632
2
0.00776899485974
3
0.00757447734116
1
0.00715690795024
2
0.0071334727899
1
0.00711153464644
1
0.00705599031567
2
0.00688177473697
1
0.006234

In [14]:
for i in range(0, len(sort_)-1):
    top_one, period = smile_fv.get_feature_pair(sort_[i])
    if abs(feature_importance_list[sort_[i]]) > 0.0:
#         print(period)
#         print(sort_[i]//380)
#         print(top_one)
        print(feature_importance_list[sort_[i]])

0.0856097596319
0.0619313347355
0.0359079801896
0.0302798403662
0.0261378252422
0.0208601773233
0.0204279146567
0.0187544526265
0.0182546364717
0.0177808755538
0.0173803621151
0.0158271993394
0.0155547313918
0.0151427704626
0.0148030336256
0.0146700705032
0.0145717008578
0.0138828194186
0.0138109642993
0.0137625996574
0.0134148727846
0.013255335347
0.0127895845135
0.0126620196511
0.0124295964513
0.0122143897299
0.0118508222603
0.0116808213473
0.0115585420845
0.0113032517726
0.0101564501269
0.0100781570064
0.00962393992729
0.0094260376558
0.00888454200505
0.00852786482959
0.00843950291447
0.00841875015699
0.00833159041766
0.00822857210907
0.00814398455121
0.00808255636995
0.00800179524595
0.00778244796632
0.00776899485974
0.00757447734116
0.00715690795024
0.0071334727899
0.00711153464644
0.00705599031567
0.00688177473697
0.00623413818531
0.00621153200848
0.00602980341486
0.00484537774409
0.00476887177971
0.00403781478674
0.00392582192883
0.00386605659786
0.00384553789213
0.003680608322


In [ ]:
# add one random column to it



X_whole = whole_df[features]


In [45]:
final_important_features = []
len_ = len(feature_importance_list)
for i in range(0, len_):
    # print(i)
    if abs(feature_importance_list[i] - feature_importance_list[len_ - 1]) > 0.0:
        final_important_features.append([i, feature_importance_list[i]])

print (final_important_features)

[[11, 0.011106130435157116], [19, 0.0048453777440859028], [21, 0.015996617998935706], [25, 0.0051145653965351204], [31, 0.036740147563287306], [35, 0.01118513208573294], [52, 0.0086229778001232476], [55, 0.0064605036587812043], [57, 0.0062481702949958281], [69, 0.01599483821177896], [70, 0.0054703584401684718], [71, 0.001641495317384207], [73, 0.0053299155184944914], [74, 0.009304312043140214], [77, 0.0031352444226438185], [81, 0.0096245657785515078], [85, 0.014789670439797082], [87, 0.0046146454705580028], [89, 0.024744560749942836], [94, 0.0053404852288432454], [99, 0.0029109395795887767], [101, 0.0080217920429866609], [105, 0.021455925634405924], [111, 0.0051684029270249628], [115, 0.0029072266464515426], [117, 0.010767506097968676], [121, 0.013310254089373646], [123, 0.0081429264865888092], [125, 0.0070104747394774524], [127, 0.0055375745646696044], [129, 0.0054464417690657917], [135, 0.0048168755220618668], [141, 0.0029396515227692863], [145, 0.031912390221590037], [146, 0.0145217

,0,1,2,3,4,5,6,7,8,9,...,210,211,212,213,214,215,216,217,218,219
0,8.867597,0.594050,17.119188,1.148048,24.559673,1.387453,30.755322,1.071753,14.020392,0.472735,...,3.348707,0.471232,6.304303,0.338737,5.670506,0.492748,10.868395,0.486781,5.261774,0.415580
0,6.512064,0.391189,13.052767,0.453070,19.531024,0.647392,25.991675,0.807966,10.616766,0.494256,...,2.709064,0.458247,5.644595,0.282355,4.451594,0.471732,9.285147,0.422140,4.888178,0.325145
0,8.443402,0.394891,16.636206,0.419559,24.570742,0.582462,32.746213,0.761213,13.723313,0.547228,...,2.435289,0.453573,5.872987,0.452762,5.346687,0.456497,10.129764,0.445514,4.826338,0.470754
0,6.457860,0.435501,13.164410,0.501497,19.855873,0.504906,26.391023,0.487157,11.769883,0.388220,...,3.123872,0.341210,6.301699,0.475484,5.176981,0.369277,10.171528,0.351213,5.021428,0.236834
0,8.443393,0.457287,16.745149,0.516371,24.995729,0.665345,33.665475,0.699957,15.135466,0.409487,...,2.952585,0.399742,7.032068,0.493865,5.862600,0.400670,11.704714,0.514762,5.880929,0.386480


In [39]:
# running linear SVM model
# we will run it for ten times, then get

from smile_fv import smile_fv 

# 1. need to use only top 5 important feature pairs from 
def run_linear_svm():
    i = 0
    average_acc = 0
    average_fscore = 0
    while i < 10:
        print (i)
        smile_fv = smile_fv(4, i, 'left_eye')
        traing_df, test_df = smile_fv.read_data()
        features = traing_df.columns[0:220]
        # print(features)
        X_train = traing_df[features]
        y_train = traing_df['Label']

        X_test = test_df[features]
        y_test = test_df['Label']

        clf_svm = LinearSVC()
        clf_svm.fit(X_train, y_train)
        y_pred_svm = clf_svm.predict(X_test)
        acc_svm = accuracy_score(y_test, y_pred_svm)
        f_score = f1_score(y_test, y_pred_svm, average='weighted')
        average_acc += acc_svm
        average_fscore += f_score
        print ("Linear SVM accuracy: ",acc_svm)
        print ("Linear SVM F-score: ",f_score)
        print (confusion_matrix(y_test, y_pred_svm))

        con_matrix = []
        con_matrix = confusion_matrix(y_test, y_pred_svm)
        P_true = con_matrix[0][0] / (con_matrix[0][0] + con_matrix[1][0])
        print ("Linear SVM accuracy of precision true: ",P_true)
        P_false = con_matrix[1][1] / (con_matrix[0][1] + con_matrix[1][1])
        print ("Linear SVM accuracy of precision false: ",P_false)

        i += 1

    average_acc = average_acc/10
    average_fscore = average_fscore/10

    print ("Average Linear SVM accuracy: ",average_acc)
    print ("Average SVM F-score: ",average_fscore)

if __name__ == '__main__':
    run_linear_svm()

0
577
66


KeyboardInterrupt: 